# WideDeep_FTTransformer

## 라이브러리 설치

In [1]:
# !pip -q install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

In [2]:
# !pip install pytorch-widedeep

In [3]:
# !pip install opencv-python-headless -> 저는 이거 없으면 엘리스에서 에러 나서 깔았습니다.

## import

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import shutil
import datetime
import os
import warnings
import random
warnings.simplefilter(action='ignore', category=FutureWarning)

import optuna

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, FunctionTransformer, QuantileTransformer, MultiLabelBinarizer
from sklearn.impute import SimpleImputer



import torch
import torch.nn as nn

# pytorch-widedeep 라이브러리 import
from pytorch_widedeep import Trainer
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, FTTransformer, WideDeep, TabFastFormer
from pytorch_widedeep.metrics import Accuracy,F1Score
from pytorch_widedeep.callbacks import EarlyStopping, ModelCheckpoint


from 평가_Metric import competition_metric, f1_score, weighted_brier_score


/opt/miniconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-05 22:43:41.730137: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-05 22:43:41.743994: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743893021.758963 1850157 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743893021.763558 1850157 cuda_blas.cc:1418] Unable to regis

## Data Load

In [2]:
train_path = f'../data/train.csv'
test_path = f'../data/test.csv'
sample_path = f'../data/sample_submission.csv'


train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])
print(train.shape, test.shape)

(126244, 34) (54412, 33)


## Data Pre-processing

In [3]:
def drop_cols_with_na(train_df, val_df):
    # 나중에 결측치 대체하면서 반영할 예정

    cat_cols_with_na = [
        '이전 총 임신 횟수',
        '이전 총 임신 성공 횟수',

        '총 생성 배아 수', ## 여기부터 100% DI
        '저장된 배아 수',
        '채취된 신선 난자 수',
        '수정 시도된 난자 수'
    ]

    numeric_cols_with_na = [
        '이식된 배아 수', ## only DI
        '미세주입(ICSI) 배아 이식 수',
        '배아 이식 후 경과일',
    ]
    train_df = train_df.drop(columns=cat_cols_with_na)
    train_df = train_df.drop(columns=numeric_cols_with_na)
    val_df = val_df.drop(columns=cat_cols_with_na)
    val_df = val_df.drop(columns=numeric_cols_with_na)
    return train_df, val_df


def 시술유형(train, test):
    train['세부 시술 유형'] = train['세부 시술 유형'].fillna("Unknown")
    test['세부 시술 유형'] = test['세부 시술 유형'].fillna("Unknown")

    def categorize_procedure(proc):
        tokens = [token.strip() for token in proc.split(",") if token.strip() and not token.strip().isdigit()]
        # 우선순위에 따른 범주화
        if tokens.count("Unknown") >= 1:
            return "Unknown"
        if tokens.count("AH") >= 1:
            return "AH"
        if tokens.count("BLASTOCYST") >= 1:
            return "BLASTOCYST"
        if tokens.count("ICSI") >= 2 or tokens.count("IVF") >= 2:
            return "2ICSI_2IVF"
        if tokens.count("IVF") >= 1 and tokens.count("ICSI") >= 1:
            return "IVF_ICSI"
        if tokens == "ICSI":
            return "ICSI"
        if tokens == "IVF":
            return "IVF"
        return ",".join(tokens) if tokens else None

    for df in [train, test]:
        df['세부 시술 유형'] = df['세부 시술 유형'].str.replace(" / ", ",")
        df['세부 시술 유형'] = df['세부 시술 유형'].str.replace(":", ",")
        df['세부 시술 유형'] = df['세부 시술 유형'].str.replace(" ", "")

    counts = train['세부 시술 유형'].value_counts()
    allowed_categories = counts[counts >= 100].index.tolist()

    # allowed_categories에 속하지 않는 값은 "Unknown"으로 대체
    train.loc[~train['세부 시술 유형'].isin(allowed_categories), '세부 시술 유형'] = "Unknown"
    test.loc[~test['세부 시술 유형'].isin(allowed_categories), '세부 시술 유형'] = "Unknown"

    train['세부 시술 유형'] = train['세부 시술 유형'].apply(categorize_procedure)
    test['세부 시술 유형'] = test['세부 시술 유형'].apply(categorize_procedure)

    train['시술유형_통합'] = train['시술 유형'].astype(str) + '_' + train['세부 시술 유형'].astype(str)
    test['시술유형_통합'] = test['시술 유형'].astype(str) + '_' + test['세부 시술 유형'].astype(str)

    drop_cols = ['시술 유형', '세부 시술 유형']
    train = train.drop(drop_cols, axis=1)
    test = test.drop(drop_cols, axis=1)

    return train, test

def 횟수_to_int(df_train, df_val):
    for col in [col for col in df_train.columns if '횟수' in col]:
        df_train[col] = df_train[col].replace({'6회 이상': '6회'})
        df_val[col] = df_val[col].replace({'6회 이상': '6회'})

        #### 일단 0으로 채움
        df_train[col] = df_train[col].fillna('0')
        df_val[col] = df_val[col].fillna('0')
        ####

        
        df_train[col] = df_train[col].str[0].astype(int)
        df_val[col] = df_val[col].str[0].astype(int)

    return df_train, df_val


def 임신_IVF(df_train, df_val):
    for col in [col for col in df_train.columns if '횟수' in col]:
        df_train[col] = df_train[col].replace({'6회 이상': '6회'})
        df_val[col] = df_val[col].replace({'6회 이상': '6회'})
        mode_value = df_train[col].mode()[0]

        df_train[col] = df_train[col].fillna(mode_value)
        df_val[col] = df_val[col].fillna(mode_value)

        # 문자열의 첫 글자를 추출 후 int형으로 변환
        df_train[col] = df_train[col].str[0].astype(int)
        df_val[col] = df_val[col].str[0].astype(int)

    df_train['임신_IVF'] = df_train['이전 총 임신 횟수'] - df_train['이전 IVF 시술 횟수']
    df_val['임신_IVF'] = df_val['이전 총 임신 횟수'] - df_val['이전 IVF 시술 횟수']
    # df_train = df_train.drop('이전 시술 횟수', axis=1)
    return df_train, df_val

def 난자기증자나이(df_train, df_test):
    mapping = {
        '만20세 이하': 20,
        '만21-25세': 25,
        '만26-30세': 30,
        '만31-35세': 35,
        '알 수 없음': 20,  # 만20세 이하와 동일하게 처리
    }
    df_train['난자 기증자 나이'] = df_train['난자 기증자 나이'].replace(mapping)
    df_test['난자 기증자 나이'] = df_test['난자 기증자 나이'].replace(mapping)
    return df_train, df_test


def 단일배아이식여부(df_train, df_val):
    df_train['단일 배아 이식 여부'] = df_train['단일 배아 이식 여부'].fillna(0)
    df_val['단일 배아 이식 여부'] = df_val['단일 배아 이식 여부'].fillna(0)
    return df_train, df_val


def 이전_총_임신_성공_횟수(train, test):
    train['이전 총 임신 횟수'] = train['이전 총 임신 횟수'].fillna(train['이전 총 임신 횟수'].mode()[0])
    test['이전 총 임신 횟수'] = test['이전 총 임신 횟수'].fillna(train['이전 총 임신 횟수'].mode()[0])

    train['이전 총 임신 성공 횟수'] = train['이전 총 임신 성공 횟수'].fillna(train['이전 총 임신 성공 횟수'].mode()[0])
    test['이전 총 임신 성공 횟수'] = test['이전 총 임신 성공 횟수'].fillna(train['이전 총 임신 성공 횟수'].mode()[0])

def 독립범주로보기(train, test):
    cols = ['총 생성 배아 수', '저장된 배아 수', '채취된 신선 난자 수', '수정 시도된 난자 수']
    for col in cols:
        train[col] = train[col].fillna('NAN')
        test[col] = test[col].fillna('NAN')


    
def label_encoding(train, test, cols):
    encoder = LabelEncoder()
    for col in cols:
        train[col] = encoder.fit_transform(train[col])
        test[col] = encoder.transform(test[col])
    return train, test

def type_to_category(train, test, cols):
    train[cols] = train[cols].astype('category')
    test[cols] = test[cols].astype('category')
    return train, test

def impute_nan(train, test):
    cat_cols_with_na = [
        '이전 총 임신 횟수',
        '이전 총 임신 성공 횟수',

        '총 생성 배아 수', ## 여기부터 100% DI
        '저장된 배아 수',
        '채취된 신선 난자 수',
        '수정 시도된 난자 수'
    ]

    numeric_cols_with_na = [
        '이식된 배아 수', ## only DI
        '미세주입(ICSI) 배아 이식 수',
        '배아 이식 후 경과일',
    ]
    cols_to_impute= cat_cols_with_na + numeric_cols_with_na
    
    for col in cols_to_impute:
        train[col] = train[col].fillna(0)
        test[col] = test[col].fillna(0)

    return train, test


def num_feature_scailing(train, test, seed=777):
    cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
    numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 확률']
    # bin_cols 들도 동일하게 스케일링


    arr_train = train[numeric_cols].to_numpy()  # DataFrame -> NumPy
    arr_train = arr_train.astype(np.float32)
    arr_test = test[numeric_cols].to_numpy()
    arr_test = arr_test.astype(np.float32)

    np.random.seed(seed)
    random.seed(seed)
    noise = (
        np.random.default_rng(0)
        .normal(0.0, 1e-5, arr_train.shape)
        .astype(arr_train.dtype)
    )
    preprocessing = QuantileTransformer(
        n_quantiles=max(min(len(train[numeric_cols]) // 30, 1000), 10),
        output_distribution='normal',
        subsample=10**9,
    ).fit(arr_train + noise)

    train[numeric_cols] = preprocessing.transform(arr_train)
    test[numeric_cols] = preprocessing.transform(arr_test)
    return train, test


def check_cols(train,val):

    cols_without_na = train.columns
    categorical_columns = ['환자 시술 당시 나이', '시술 유형', '세부 시술 유형', '이전 IVF 시술 횟수',
       '이전 DI 시술 횟수', '이전 총 임신 횟수', '이전 총 임신 성공 횟수', '총 생성 배아 수', '저장된 배아 수',
       '해동된 배아 수', '채취된 신선 난자 수', '수정 시도된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이',
       '정자 기증자 나이']

    numeric_columns = ['이식된 배아 수','미세주입(ICSI) 배아 이식 수','배아 이식 후 경과일']

    bool_features = ['배란 자극 시술 여부', '단일 배아 이식 여부', '불임 원인 - 난관 질환',
    '불임 원인 - 배란 장애', '불임 원인 - 남성 요인', '불임 원인 - 자궁내막증', '불임 원인 - 불명확', '해동 난자 사용 여부',
    '신선 난자 사용 여부', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '착상 전 PGD 시행 여부', '착상 전 PGS 시행 여부']
    
    categorical_columns_cleaned = [col for col in categorical_columns if col in cols_without_na]
    numeric_columns_cleaned = [col for col in numeric_columns if col in cols_without_na]
    bool_features_cleaned = [col for col in bool_features if col in cols_without_na]
    
    # print("Categorical columns:", categorical_columns_cleaned, "\n count", len(categorical_columns_cleaned))
    # print("Numeric columns:", numeric_columns_cleaned, "\n count", len(numeric_columns_cleaned))
    # print("Boolean columns:", bool_features_cleaned, "\n count", len(bool_features_cleaned))

    return categorical_columns_cleaned, numeric_columns_cleaned, bool_features_cleaned


def drop_single_value_columns(df_train, df_test):
    cols_to_drop = [col for col in df_train.columns if df_train[col].nunique() == 1]
    return df_train.drop(columns=cols_to_drop), df_test.drop(columns=cols_to_drop)

In [4]:
def all_process(train, val):
    # train, val = drop_cols_with_na(train, val)

    # 기본 전처리 단계
    train, val = 횟수_to_int(train, val)

    train, val = 시술유형(train, val)
    # train, val = 임신_IVF(train, val)

    train, val = 단일배아이식여부(train, val)
    이전_총_임신_성공_횟수(train, val)
    독립범주로보기(train, val)

    
    cols_to_encoding = [
        "환자 시술 당시 나이",
        # "클리닉 내 총 시술 횟수",
        # "IVF 시술 횟수",
        # "DI 시술 횟수",
        # "총 임신 횟수",
        # "IVF 임신 횟수",
        # "DI 임신 횟수",
        # "총 출산 횟수",
        # "IVF 출산 횟수",
        # "DI 출산 횟수",
        "난자 출처",
        "정자 출처",
        "난자 기증자 나이",
        "정자 기증자 나이",
        '시술유형_통합',

        '해동된 배아 수', # 원래 int였는데 범주형으로 바뀜
        '총 생성 배아 수',
        '저장된 배아 수', 
        '채취된 신선 난자 수', 
        '수정 시도된 난자 수'
        

    ]
    
    train, val = label_encoding(train, val, cols=cols_to_encoding)
    train, val = type_to_category(train, val, cols=cols_to_encoding)

    train, val = impute_nan(train, val)
    train, val = num_feature_scailing(train, val)

    train, val = drop_single_value_columns(train, val)

    return train, val

train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])

train, test = all_process(train, test)

cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 확률']

print(f'수치형 변수: {len(numeric_cols)}개 \n{numeric_cols}')
print(f'범주형 변수: {len(cat_cols)}개 \n{cat_cols}')
print(train.shape, test.shape)

/tmp/ipykernel_1850157/641393060.py:180: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]


수치형 변수: 21개 
['배란 자극 시술 여부', '단일 배아 이식 여부', '불임 원인 - 난관 질환', '불임 원인 - 배란 장애', '불임 원인 - 남성 요인', '불임 원인 - 자궁내막증', '불임 원인 - 불명확', '이전 IVF 시술 횟수', '이전 DI 시술 횟수', '이전 총 임신 횟수', '이전 총 임신 성공 횟수', '이식된 배아 수', '미세주입(ICSI) 배아 이식 수', '해동 난자 사용 여부', '신선 난자 사용 여부', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '착상 전 PGD 시행 여부', '착상 전 PGS 시행 여부', '배아 이식 후 경과일']
범주형 변수: 11개 
['환자 시술 당시 나이', '총 생성 배아 수', '저장된 배아 수', '해동된 배아 수', '채취된 신선 난자 수', '수정 시도된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '시술유형_통합']
(126244, 33) (54412, 32)


/tmp/ipykernel_1850157/1834065377.py:56: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]


## FTT

In [5]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU


In [ ]:
import time
start_time = time.time()



# 교차 검증 설정: seed_list를 [333, 777], n_splits=5
seed_list = [333, 777]
n_splits = 5

total_score, total_wbs_score, total_f1_score = [], [], []
test_preds = []


# 교차 검증 시작
for seed in seed_list:
    set_seed(seed)
    # train, test 불러오기
    train = pd.read_csv(train_path).drop(columns=["ID"])
    test = pd.read_csv(test_path).drop(columns=["ID"])

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    scores, wbs_scores, f1_scores = [], [], []
    
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train)):
        # Fold 데이터 생성
        fold_train, fold_valid = train.iloc[train_idx], train.iloc[valid_idx]
        fold_train2 = fold_train.copy()
        fold_test = test.copy()  # test 데이터는 별도 사용
        
        # 전처리 
        fold_train, fold_valid = all_process(fold_train, fold_valid)
        _, fold_test = all_process(fold_train2, fold_test)

        # 범주형, 연속형 열 구분분
        categorical_cols = [col for col in fold_train.columns if pd.api.types.is_categorical_dtype(fold_train[col])]
        continuous_cols = [col for col in fold_train.columns if col not in cat_cols and col != '임신 성공 확률']


        # Wide 부분: 원-핫 인코딩 (crossed_cols 옵션도 사용할 수 있음)
        wide_cols = categorical_cols  # Wide 모델에는 범주형 변수를 원-핫 인코딩으로 처리
        crossed_cols = []  # 필요시 두 개 이상의 컬럼을 교차시켜 상호작용 feature 생성

        # WidePreprocessor (원-핫 인코딩, crossed_cols 사용 가능)
        wide_preprocessor = WidePreprocessor(wide_cols=wide_cols, crossed_cols=crossed_cols)

        # Deep 부분: 임베딩 + 연속형 변수 처리
        tab_preprocessor = TabPreprocessor(embed_cols=categorical_cols, continuous_cols=continuous_cols)

        # 전처리: 각 Fold 별로 Wide & Deep 데이터 생성
        X_wide_train = wide_preprocessor.fit_transform(fold_train)
        X_wide_valid = wide_preprocessor.transform(fold_valid)
        X_wide_test = wide_preprocessor.transform(fold_test)
        X_tab_train = tab_preprocessor.fit_transform(fold_train)
        X_tab_valid = tab_preprocessor.transform(fold_valid)
        X_tab_test = tab_preprocessor.transform(fold_test)

        # Target 값: 정수형 (0,1)
        y_train = fold_train['임신 성공 확률'].astype(int).values
        y_valid = fold_valid['임신 성공 확률'].astype(int).values

        # Wide 모델: 입력 차원은 원-핫 인코딩된 피처 수####
        wide_model = Wide(input_dim=int(X_wide_train.max().item()), pred_dim=1)

        
        # Deep 모델: FTTransformer 사용
        tab_model = FTTransformer(
            column_idx=tab_preprocessor.column_idx,  # 각 컬럼의 인덱스 정보
            cat_embed_input=tab_preprocessor.cat_embed_input,  # (컬럼명, 고유값 수, 임베딩 차원)

            continuous_cols=continuous_cols,  # 연속형 변수 리스트

            # ▶ 범주형 임베딩 관련 설정
            cat_embed_dropout=0.0,  # 범주형 임베딩 드롭아웃
            # 후보 값: 0.0 (기본), 0.1, 0.2, 0.3


            cat_embed_activation=None,  # 범주형 임베딩 활성화 함수
            # 후보 값: None (기본), 'relu', 'leaky_relu', 'tanh', 'gelu'


            # ▶ 연속형 변수 관련 설정
            cont_norm_layer="batchnorm",  # 연속형 변수 정규화 방식
            # 후보 값: None (사용 안함), 'batchnorm' (기본 추천), 'layernorm'

            embed_continuous_method="standard",  # 연속형 변수 임베딩 방식
            # 후보 값: 'standard' (기본), 'periodic', 'piecewise'

            cont_embed_dropout=0.0,  # 연속형 임베딩 드롭아웃
            # 후보 값: 0.0 (기본), 0.1, 0.2, 0.3

            cont_embed_activation='relu',  # 연속형 임베딩 활성화 함수
            # 후보 값: None (기본), 'relu', 'leaky_relu', 'tanh', 'gelu'

            quantization_setup=None,  # 'piecewise' 방식에서 구간 경계 지정
            # 후보 값: None (기본), {"age": [20, 30, 40], "income": [10000, 30000, 60000]}

            # ▶ 기타 임베딩 관련
            full_embed_dropout=False,  # 전체 임베딩을 dropout할지 여부
            # 후보 값: False (기본), True (더 강한 정규화)

            # ▶ FTTransformer 구조 설정
            input_dim=64,  # 임베딩 차원 수 (카테고리 + 연속형 임베딩 포함)
            # 후보 값: 32, 64 (기본), 128


            n_heads=8,  # Attention 헤드 수
            # 후보 값: 4, 8 (기본), 16


            n_blocks=4,  # Transformer block 수
            # 후보 값: 2 (얕은 모델), 4 (기본), 6 (깊은 모델)

            attn_dropout=0.2,  # Attention dropout
            # 후보 값: 0.0, 0.1, 0.2 (기본), 0.3

####

            transformer_activation="reglu",  # Transformer 내부 활성화 함수
            # 후보 값: 'relu', 'gelu', 'leaky_relu', 'tanh', 'geglu', 'reglu' (기본)

            # ▶ MLP 설정 (선택 사항)
            mlp_hidden_dims=[64, 32],  # MLP 은닉층 크기
            # 후보 값: [64, 32] (기본), [128, 64], [256, 128], None (사용 안 함)

            mlp_activation="relu",  # MLP 활성화 함수
            # 후보 값: 'relu' (기본), 'leaky_relu', 'tanh', 'gelu'

            mlp_dropout=0.1,  # MLP dropout
            # 후보 값: 0.0, 0.1 (기본), 0.3

            mlp_batchnorm=False,  # MLP에 배치 정규화 적용 여부
            # 후보 값: False (기본), True

            mlp_batchnorm_last=False,  # MLP 마지막 층에도 BN 적용 여부
            # 후보 값: False (기본), True

        )
        
        # Wide & Deep 모델 결합
        model = WideDeep(wide=wide_model, deeptabular=tab_model)
        
        # 옵티마이저 및 학습률 스케줄러 설정
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  # Set your desired learning rate here
        

        # ✅ Training 데이터를 딕셔너리 형태로 생성
        X_train = {
            "X_wide": X_wide_train,
            "X_tab": X_tab_train,
            "target": y_train
        }

        # ✅ Validation 데이터를 딕셔너리로 전달 (X_val 방식 사용)
        X_val = {
            "X_wide": X_wide_valid,
            "X_tab": X_tab_valid,
            "target": y_valid
        }
        
        # EarlyStopping 콜백 설정 (patience=5, min_delta=0.001)
        early_stopping = EarlyStopping(
            monitor="val_loss", 
            patience=5, 
            min_delta=0.001, 
            verbose=1
        )
        
        # 현재 날짜/시간을 포함한 파일 이름 생성
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        model_path = f"saved_models/best_model_{timestamp}.pt"
        
        # ✅ 2. ModelCheckpoint: 최상의 모델을 자동 저장
        model_checkpoint = ModelCheckpoint(
            filepath=model_path,  # 모델 저장 경로
            monitor="val_loss",        # 감시할 지표 ('val_loss' 또는 'val_acc')
            save_best_only=True        # 가장 좋은 성능의 모델만 저장
        )



        # 클래스 불균형을 반영하기 위한 변수
        y_train_binary = (y_train > 0.5).astype(int)
        positive_count = y_train_binary.sum()
        negative_count = len(y_train_binary) - positive_count
        
        pos_weight = torch.tensor(negative_count / positive_count)  # 기본
        
        # sqrt 적용:
        # pos_weight = torch.sqrt(torch.tensor(negative_count / positive_count))
        
        # # log 적용:
        # pos_weight = torch.log1p(torch.tensor(negative_count / positive_count))
        # ✅ custom loss: BCEWithLogitsLoss (pos_weight 사용 안 함)
        custom_loss = nn.BCEWithLogitsLoss(pos_weight=pos_weight) # 클래스 비율을 반영

        # Trainer 생성: objective "binary"로 설정, 평가 지표로 Accuracy, AUROC, F1Score 사용
        trainer = Trainer(
            model=model, 
            objective="binary", ### 이거를 regression으로 바꾸는 게 나은 것인지를 모르겠음
            custom_loss_function=custom_loss,  # 기본 손실 함수 사용 (필요 시 사용자 정의 함수 가능)
            optimizers=optimizer,       # 옵티마이저 (기본값: Adam)
            initializers=None,          # 가중치 초기화 없음 (기본값)
            callbacks=[early_stopping, model_checkpoint],  # 조기 종료 (3 에포크 동안 개선 없으면 종료)
            metrics=[Accuracy()],
            verbose=0,                  # 학습 로그 출력 (기본값: 1)
            seed=seed                     # 랜덤 시드 설정 (기본값: 1)
        )
        
        # 학습: validation 데이터는 별도의 인자로 전달
        trainer.fit(
            X_train=X_train,  # ✅ Training 데이터를 딕셔너리로 
            n_epochs=100,  ####
            batch_size=1024, 
            X_val=X_val  # ✅ Validation 데이터를 딕셔너리로 전달
        )
        
        
        
        # 모델 학습 후 Validation 예측 코드:
        # 이미 확률이 계산되어 있는 컬럼을 사용합니다.
        valid_probs = trainer.predict_proba(X_wide=X_wide_valid, X_tab=X_tab_valid)[:,1] ####
        # valid_probs = trainer.predict(X_wide=X_wide_valid, X_tab=X_tab_valid)
        # valid_probs = np.clip(valid_probs, 0, 1) ##### regression 일때
        valid_pred = (valid_probs > 0.5).astype(int)

        # DataFrame 으로 변환
        valid_probs_df = pd.DataFrame({'prob': valid_probs})
            
        

        # 실제 정답 
        y_valid = fold_valid['임신 성공 확률'].values.astype(int)
        

        
        # 대회 평가 지표
        fold_score = competition_metric(y_valid, valid_probs)
        fold_wbs = weighted_brier_score(y_valid, valid_probs)
        fold_f1 = f1_score(y_valid, valid_probs)

        
        print(f"Seed[{seed:<3}] Fold {fold + 1} | SCORE: {fold_score:.7f} | WBS: {fold_wbs:.7f} | F1: {fold_f1:.7f}")

        
        scores.append(fold_score)
        wbs_scores.append(fold_wbs)
        f1_scores.append(fold_f1)
        
        total_score.append(fold_score)
        total_wbs_score.append(fold_wbs)
        total_f1_score.append(f1_scores)
        
        # Test 데이터 예측 (각 fold의 모델로 예측한 결과 저장)
        test_pred = trainer.predict_proba(X_wide=X_wide_test, X_tab=X_tab_test)[:,1]
        # test_pred = trainer.predict(X_wide=X_wide_test, X_tab=X_tab_test)
        # test_pred = np.clip(test_pred, 0, 1) ##### regression 일때
        test_preds.append(test_pred)
    
    # Fold 별 평균 성능 출력
    avg_score = np.mean(scores)
    avg_wbs = np.mean(wbs_scores)
    avg_f1 = np.mean(f1_scores)
    
    print("-" * 80)
    print(f"Seed[{seed:<3}] Average Metrics | SCORE: {avg_score:.7f} | WBS: {avg_wbs:.7f} | F1: {avg_f1:.7f}")
    print("-" * 80)

# 전체 Validation 평균 성능 출력
val_score = np.mean(total_score)
val_wbs = np.mean(total_wbs_score)
val_f1 = np.mean(total_f1_score)


print("-" * 80)
print(f"Validation Average Metrics | SCORE: {val_score:.7f} | WBS: {val_wbs:.7f} | F1: {val_f1:.7f}")
print("-" * 80)

finish_time = time.time()
total_time = finish_time - start_time 

print(total_time)

/tmp/ipykernel_1850157/641393060.py:180: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
/tmp/ipykernel_1850157/641393060.py:180: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
/tmp/ipykernel_1850157/166562470.py:38: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  categorical_cols = [col for col in fold_train.columns if pd.api.types.is_categorical_dtype(fold_train[col])]
/home/ubuntu/.local/lib/python3.12/site-packages/pytorch_widedeep/preprocessing/tab_preprocessor.py:364: UserWarning: Continuous columns wil

Best Epoch: 20. Best val_loss: 0.65658
Model weights restored to best epoch: 25


predict: 100%|██████████| 25/25 [00:03<00:00,  6.42it/s]


Seed[333] Fold 1 | SCORE: 0.5880548 | WBS: 0.7659362 | F1: 0.4101734


  0%|          | 0/54 [00:00<?, ?it/s]/opt/miniconda/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: This process (pid=1850157) is multi-threaded, use of fork() may lead to deadlocks in the child.
  self.pid = os.fork()
predict: 100%|██████████| 54/54 [00:04<00:00, 11.91it/s]
/tmp/ipykernel_1850157/641393060.py:180: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
/tmp/ipykernel_1850157/641393060.py:180: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
/tmp/ipykernel_1850157/166562470.py:38: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use 

KeyboardInterrupt: 

- Validation Average Metrics | SCORE: 0.6339700 | WBS: 0.7970266 | F1: 0.4709135

- 열 제거: 0.3477212
- 열 모두 살리고 0으로 채우기 : 0.4411126 (object: binary)
- 열 모두 살리고 0으로 채우기 : 0.4041905 (object: regression)

In [ ]:
tmp_prediction = pd.DataFrame({'FTT': np.mean(test_preds, axis=0)})

## TabFastFormer

In [ ]:
train_path = f'../data/train.csv'
test_path = f'../data/test.csv'
sample_path = f'../data/sample_submission.csv'


train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])
print(train.shape, test.shape)

In [ ]:
def drop_cols_with_na(train_df, val_df):
    # 나중에 결측치 대체하면서 반영할 예정

    cat_cols_with_na = [
        '이전 총 임신 횟수',
        '이전 총 임신 성공 횟수',

        '총 생성 배아 수', ## 여기부터 100% DI
        '저장된 배아 수',
        '채취된 신선 난자 수',
        '수정 시도된 난자 수'
    ]

    numeric_cols_with_na = [
        '이식된 배아 수', ## only DI
        '미세주입(ICSI) 배아 이식 수',
        '배아 이식 후 경과일',
    ]
    train_df = train_df.drop(columns=cat_cols_with_na)
    train_df = train_df.drop(columns=numeric_cols_with_na)
    val_df = val_df.drop(columns=cat_cols_with_na)
    val_df = val_df.drop(columns=numeric_cols_with_na)
    return train_df, val_df


def 시술유형(train, test):
    train['세부 시술 유형'] = train['세부 시술 유형'].fillna("Unknown")
    test['세부 시술 유형'] = test['세부 시술 유형'].fillna("Unknown")

    def categorize_procedure(proc):
        tokens = [token.strip() for token in proc.split(",") if token.strip() and not token.strip().isdigit()]
        # 우선순위에 따른 범주화
        if tokens.count("Unknown") >= 1:
            return "Unknown"
        if tokens.count("AH") >= 1:
            return "AH"
        if tokens.count("BLASTOCYST") >= 1:
            return "BLASTOCYST"
        if tokens.count("ICSI") >= 2 or tokens.count("IVF") >= 2:
            return "2ICSI_2IVF"
        if tokens.count("IVF") >= 1 and tokens.count("ICSI") >= 1:
            return "IVF_ICSI"
        if tokens == "ICSI":
            return "ICSI"
        if tokens == "IVF":
            return "IVF"
        return ",".join(tokens) if tokens else None

    for df in [train, test]:
        df['세부 시술 유형'] = df['세부 시술 유형'].str.replace(" / ", ",")
        df['세부 시술 유형'] = df['세부 시술 유형'].str.replace(":", ",")
        df['세부 시술 유형'] = df['세부 시술 유형'].str.replace(" ", "")

    counts = train['세부 시술 유형'].value_counts()
    allowed_categories = counts[counts >= 100].index.tolist()

    # allowed_categories에 속하지 않는 값은 "Unknown"으로 대체
    train.loc[~train['세부 시술 유형'].isin(allowed_categories), '세부 시술 유형'] = "Unknown"
    test.loc[~test['세부 시술 유형'].isin(allowed_categories), '세부 시술 유형'] = "Unknown"

    train['세부 시술 유형'] = train['세부 시술 유형'].apply(categorize_procedure)
    test['세부 시술 유형'] = test['세부 시술 유형'].apply(categorize_procedure)

    train['시술유형_통합'] = train['시술 유형'].astype(str) + '_' + train['세부 시술 유형'].astype(str)
    test['시술유형_통합'] = test['시술 유형'].astype(str) + '_' + test['세부 시술 유형'].astype(str)

    drop_cols = ['시술 유형', '세부 시술 유형']
    train = train.drop(drop_cols, axis=1)
    test = test.drop(drop_cols, axis=1)

    return train, test

def 횟수_to_int(df_train, df_val):
    for col in [col for col in df_train.columns if '횟수' in col]:
        df_train[col] = df_train[col].replace({'6회 이상': '6회'})
        df_val[col] = df_val[col].replace({'6회 이상': '6회'})

        #### 일단 0으로 채움
        df_train[col] = df_train[col].fillna('0')
        df_val[col] = df_val[col].fillna('0')
        ####

        
        df_train[col] = df_train[col].str[0].astype(int)
        df_val[col] = df_val[col].str[0].astype(int)

    return df_train, df_val


def 임신_IVF(df_train, df_val):
    for col in [col for col in df_train.columns if '횟수' in col]:
        df_train[col] = df_train[col].replace({'6회 이상': '6회'})
        df_val[col] = df_val[col].replace({'6회 이상': '6회'})
        mode_value = df_train[col].mode()[0]

        df_train[col] = df_train[col].fillna(mode_value)
        df_val[col] = df_val[col].fillna(mode_value)

        # 문자열의 첫 글자를 추출 후 int형으로 변환
        df_train[col] = df_train[col].str[0].astype(int)
        df_val[col] = df_val[col].str[0].astype(int)

    df_train['임신_IVF'] = df_train['이전 총 임신 횟수'] - df_train['이전 IVF 시술 횟수']
    df_val['임신_IVF'] = df_val['이전 총 임신 횟수'] - df_val['이전 IVF 시술 횟수']
    # df_train = df_train.drop('이전 시술 횟수', axis=1)
    return df_train, df_val

def 난자기증자나이(df_train, df_test):
    mapping = {
        '만20세 이하': 20,
        '만21-25세': 25,
        '만26-30세': 30,
        '만31-35세': 35,
        '알 수 없음': 20,  # 만20세 이하와 동일하게 처리
    }
    df_train['난자 기증자 나이'] = df_train['난자 기증자 나이'].replace(mapping)
    df_test['난자 기증자 나이'] = df_test['난자 기증자 나이'].replace(mapping)
    return df_train, df_test


def 단일배아이식여부(df_train, df_val):
    df_train['단일 배아 이식 여부'] = df_train['단일 배아 이식 여부'].fillna(0)
    df_val['단일 배아 이식 여부'] = df_val['단일 배아 이식 여부'].fillna(0)
    return df_train, df_val


def 이전_총_임신_성공_횟수(train, test):
    train['이전 총 임신 횟수'] = train['이전 총 임신 횟수'].fillna(train['이전 총 임신 횟수'].mode()[0])
    test['이전 총 임신 횟수'] = test['이전 총 임신 횟수'].fillna(train['이전 총 임신 횟수'].mode()[0])

    train['이전 총 임신 성공 횟수'] = train['이전 총 임신 성공 횟수'].fillna(train['이전 총 임신 성공 횟수'].mode()[0])
    test['이전 총 임신 성공 횟수'] = test['이전 총 임신 성공 횟수'].fillna(train['이전 총 임신 성공 횟수'].mode()[0])

def 독립범주로보기(train, test):
    cols = ['총 생성 배아 수', '저장된 배아 수', '채취된 신선 난자 수', '수정 시도된 난자 수']
    for col in cols:
        train[col] = train[col].fillna('NAN')
        test[col] = test[col].fillna('NAN')


    
def label_encoding(train, test, cols):
    encoder = LabelEncoder()
    for col in cols:
        train[col] = encoder.fit_transform(train[col])
        test[col] = encoder.transform(test[col])
    return train, test

def type_to_category(train, test, cols):
    train[cols] = train[cols].astype('category')
    test[cols] = test[cols].astype('category')
    return train, test

def impute_nan(train, test):
    cat_cols_with_na = [
        '이전 총 임신 횟수',
        '이전 총 임신 성공 횟수',

        '총 생성 배아 수', ## 여기부터 100% DI
        '저장된 배아 수',
        '채취된 신선 난자 수',
        '수정 시도된 난자 수'
    ]

    numeric_cols_with_na = [
        '이식된 배아 수', ## only DI
        '미세주입(ICSI) 배아 이식 수',
        '배아 이식 후 경과일',
    ]
    cols_to_impute= cat_cols_with_na + numeric_cols_with_na
    
    for col in cols_to_impute:
        train[col] = train[col].fillna(0)
        test[col] = test[col].fillna(0)

    return train, test


def num_feature_scailing(train, test, seed=777):
    cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
    numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 확률']
    # bin_cols 들도 동일하게 스케일링


    arr_train = train[numeric_cols].to_numpy()  # DataFrame -> NumPy
    arr_train = arr_train.astype(np.float32)
    arr_test = test[numeric_cols].to_numpy()
    arr_test = arr_test.astype(np.float32)

    np.random.seed(seed)
    random.seed(seed)
    noise = (
        np.random.default_rng(0)
        .normal(0.0, 1e-5, arr_train.shape)
        .astype(arr_train.dtype)
    )
    preprocessing = QuantileTransformer(
        n_quantiles=max(min(len(train[numeric_cols]) // 30, 1000), 10),
        output_distribution='normal',
        subsample=10**9,
    ).fit(arr_train + noise)

    train[numeric_cols] = preprocessing.transform(arr_train)
    test[numeric_cols] = preprocessing.transform(arr_test)
    return train, test


def check_cols(train,val):

    cols_without_na = train.columns
    categorical_columns = ['환자 시술 당시 나이', '시술 유형', '세부 시술 유형', '이전 IVF 시술 횟수',
       '이전 DI 시술 횟수', '이전 총 임신 횟수', '이전 총 임신 성공 횟수', '총 생성 배아 수', '저장된 배아 수',
       '해동된 배아 수', '채취된 신선 난자 수', '수정 시도된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이',
       '정자 기증자 나이']

    numeric_columns = ['이식된 배아 수','미세주입(ICSI) 배아 이식 수','배아 이식 후 경과일']

    bool_features = ['배란 자극 시술 여부', '단일 배아 이식 여부', '불임 원인 - 난관 질환',
    '불임 원인 - 배란 장애', '불임 원인 - 남성 요인', '불임 원인 - 자궁내막증', '불임 원인 - 불명확', '해동 난자 사용 여부',
    '신선 난자 사용 여부', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '착상 전 PGD 시행 여부', '착상 전 PGS 시행 여부']
    
    categorical_columns_cleaned = [col for col in categorical_columns if col in cols_without_na]
    numeric_columns_cleaned = [col for col in numeric_columns if col in cols_without_na]
    bool_features_cleaned = [col for col in bool_features if col in cols_without_na]
    
    # print("Categorical columns:", categorical_columns_cleaned, "\n count", len(categorical_columns_cleaned))
    # print("Numeric columns:", numeric_columns_cleaned, "\n count", len(numeric_columns_cleaned))
    # print("Boolean columns:", bool_features_cleaned, "\n count", len(bool_features_cleaned))

    return categorical_columns_cleaned, numeric_columns_cleaned, bool_features_cleaned


def drop_single_value_columns(df_train, df_test):
    cols_to_drop = [col for col in df_train.columns if df_train[col].nunique() == 1]
    return df_train.drop(columns=cols_to_drop), df_test.drop(columns=cols_to_drop)

In [ ]:
def all_process(train, val):
    # train, val = drop_cols_with_na(train, val)

    # 기본 전처리 단계
    train, val = 횟수_to_int(train, val)

    train, val = 시술유형(train, val)
    # train, val = 임신_IVF(train, val)

    train, val = 단일배아이식여부(train, val)
    이전_총_임신_성공_횟수(train, val)
    독립범주로보기(train, val)
    
    cols_to_encoding = [
        "환자 시술 당시 나이",
        # "클리닉 내 총 시술 횟수",
        # "IVF 시술 횟수",
        # "DI 시술 횟수",
        # "총 임신 횟수",
        # "IVF 임신 횟수",
        # "DI 임신 횟수",
        # "총 출산 횟수",
        # "IVF 출산 횟수",
        # "DI 출산 횟수",
        "난자 출처",
        "정자 출처",
        "난자 기증자 나이",
        "정자 기증자 나이",
        '시술유형_통합',

        '해동된 배아 수', # 원래 int였는데 범주형으로 바뀜
        '총 생성 배아 수',
        '저장된 배아 수', 
        '채취된 신선 난자 수', 
        '수정 시도된 난자 수'
        

    ]
    
    train, val = label_encoding(train, val, cols=cols_to_encoding)
    train, val = type_to_category(train, val, cols=cols_to_encoding)

    train, val = impute_nan(train, val)
    train, val = num_feature_scailing(train, val)

    train, val = drop_single_value_columns(train, val)

    return train, val

train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])

train, test = all_process(train, test)

cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 확률']

print(f'수치형 변수: {len(numeric_cols)}개 \n{numeric_cols}')
print(f'범주형 변수: {len(cat_cols)}개 \n{cat_cols}')
print(train.shape, test.shape)

In [ ]:
import time
start_time = time.time()



# 교차 검증 설정: seed_list를 [333] 하나만 사용, n_splits=3
seed_list = [333]
n_splits = 5

total_score, total_wbs_score, total_f1_score = [], [], []
test_preds = []





# 교차 검증 시작
for seed in seed_list:
    set_seed(seed)
    # train, test 불러오기
    train = pd.read_csv(train_path).drop(columns=["ID"])
    test = pd.read_csv(test_path).drop(columns=["ID"])

    kf = KFold(n_splits=5, shuffle=True, random_state=seed)
    scores, wbs_scores, f1_scores = [], [], []
    
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train)):
        # Fold 데이터 생성
        fold_train, fold_valid = train.iloc[train_idx], train.iloc[valid_idx]
        fold_train2 = fold_train.copy()
        fold_test = test.copy()  # test 데이터는 별도 사용
        
        # 전처리 
        fold_train, fold_valid = all_process(fold_train, fold_valid)
        _, fold_test = all_process(fold_train2, fold_test)

        # 범주형, 연속형 열 구분분
        categorical_cols = [col for col in fold_train.columns if pd.api.types.is_categorical_dtype(fold_train[col])]
        continuous_cols = [col for col in fold_train.columns if col not in cat_cols and col != '임신 성공 확률']


        # Wide 부분: 원-핫 인코딩 (crossed_cols 옵션도 사용할 수 있음)
        wide_cols = categorical_cols  # Wide 모델에는 범주형 변수를 원-핫 인코딩으로 처리
        crossed_cols = []  # 필요시 두 개 이상의 컬럼을 교차시켜 상호작용 feature 생성

        # WidePreprocessor (원-핫 인코딩, crossed_cols 사용 가능)
        wide_preprocessor = WidePreprocessor(wide_cols=wide_cols, crossed_cols=crossed_cols)

        # Deep 부분: 임베딩 + 연속형 변수 처리
        tab_preprocessor = TabPreprocessor(embed_cols=categorical_cols, continuous_cols=continuous_cols)

        # 전처리: 각 Fold 별로 Wide & Deep 데이터 생성
        X_wide_train = wide_preprocessor.fit_transform(fold_train)
        X_wide_valid = wide_preprocessor.transform(fold_valid)
        X_wide_test = wide_preprocessor.transform(fold_test)
        X_tab_train = tab_preprocessor.fit_transform(fold_train)
        X_tab_valid = tab_preprocessor.transform(fold_valid)
        X_tab_test = tab_preprocessor.transform(fold_test)

        # Target 값: 정수형 (0,1)
        y_train = fold_train['임신 성공 확률'].astype(int).values
        y_valid = fold_valid['임신 성공 확률'].astype(int).values

        # Wide 모델: 입력 차원은 원-핫 인코딩된 피처 수####
        wide_model = Wide(input_dim=int(X_wide_train.max().item()), pred_dim=1)


        
        # Deep 모델: TabFastFormer 사용
        tab_model = TabFastFormer(
            column_idx=tab_preprocessor.column_idx,  # 각 컬럼의 인덱스 정보
            cat_embed_input=tab_preprocessor.cat_embed_input,  # (컬럼명, 고유값 수, 임베딩 차원)

            continuous_cols=continuous_cols,  # 연속형 변수 리스트

            # ▶ 범주형 임베딩 설정
            cat_embed_dropout=0.0,  # 범주형 임베딩 드롭아웃
            # 후보 값: 0.0 (기본), 0.1, 0.2, 0.3

            use_cat_bias=False,  # 범주형 임베딩에 bias 사용 여부
            # 후보 값: False (기본), True

            cat_embed_activation=None,  # 범주형 임베딩 활성화 함수
            # 후보 값: None (기본), 'relu', 'tanh', 'leaky_relu', 'gelu'


            # ▶ 연속형 변수 설정
            cont_norm_layer='batchnorm',  # 연속형 변수 정규화 방식
            # 후보 값: None, 'batchnorm', 'layernorm'

            embed_continuous_method='standard',  # 연속형 변수 임베딩 방식
            # 후보 값: 'standard' (기본), 'piecewise', 'periodic'

            cont_embed_dropout=0.0,  # 연속형 임베딩 드롭아웃
            # 후보 값: 0.0 (기본), 0.1, 0.3

            cont_embed_activation='relu',  # 연속형 임베딩 활성화 함수
            # 후보 값: None (기본), 'relu', 'leaky_relu', 'gelu'

            quantization_setup=None,  # piecewise 임베딩 시 구간 설정
            # 후보 값: None (기본), {'age': [20, 30, 40], 'income': [10000, 30000]}


            full_embed_dropout=False,  # 전체 임베딩 dropout 여부
            # 후보 값: False (기본), True

            # ▶ FastFormer 구조 설정
            input_dim=32,  # 임베딩 차원
            # 후보 값: 32 (기본), 64, 128

            n_heads=8,  # 어텐션 헤드 개수
            # 후보 값: 4, 8 (기본), 16

            use_bias=False,  # QKV에 bias 적용 여부
            # 후보 값: False (기본), True

            n_blocks=4,  # FastFormer 블록 수
            # 후보 값: 2, 4 (기본), 6

            attn_dropout=0.1,  # 어텐션 드롭아웃
            # 후보 값: 0.0, 0.1 (기본), 0.2


            transformer_activation="relu",  # Transformer 내부 활성화 함수
            # 후보 값: 'relu' (기본), 'gelu', 'geglu', 'reglu', 'tanh', 'leaky_relu'

            # ▶ MLP 추가 설정 (선택적)
            mlp_hidden_dims=[64, 32],  # MLP 은닉층 크기
            # 후보 값: [64, 32] (기본), [128, 64], None (사용 안 함)

            mlp_activation="relu",  # MLP 활성화 함수
            # 후보 값: 'relu' (기본), 'gelu', 'leaky_relu', 'tanh'

            mlp_dropout=0.1,  # MLP dropout
            # 후보 값: 0.0, 0.1 (기본), 0.3

            mlp_batchnorm=False,  # MLP에 배치 정규화 사용 여부
            # 후보 값: False (기본), True

            mlp_batchnorm_last=False,  # 마지막 층에도 배치 정규화
            # 후보 값: False (기본), True

            mlp_linear_first=True  # MLP 연산 순서
            # 후보 값: True (기본), False
        )
        
        # Wide & Deep 모델 결합
        model = WideDeep(wide=wide_model, deeptabular=tab_model)
        
        # 옵티마이저 및 학습률 스케줄러 설정
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  # Set your desired learning rate here
        

        # ✅ Training 데이터를 딕셔너리 형태로 생성
        X_train = {
            "X_wide": X_wide_train,
            "X_tab": X_tab_train,
            "target": y_train
        }

        # ✅ Validation 데이터를 딕셔너리로 전달 (X_val 방식 사용)
        X_val = {
            "X_wide": X_wide_valid,
            "X_tab": X_tab_valid,
            "target": y_valid
        }
        
        # EarlyStopping 콜백 설정 (patience=5, min_delta=0.001)
        early_stopping = EarlyStopping(
            monitor="val_loss", 
            patience=5, 
            min_delta=0.001, 
            verbose=1
        )
        
        # 현재 날짜/시간을 포함한 파일 이름 생성
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        model_path = f"saved_models/best_model_{timestamp}.pt"
        
        # ✅ 2. ModelCheckpoint: 최상의 모델을 자동 저장
        model_checkpoint = ModelCheckpoint(
            filepath=model_path,  # 모델 저장 경로
            monitor="val_loss",        # 감시할 지표 ('val_loss' 또는 'val_acc')
            save_best_only=True        # 가장 좋은 성능의 모델만 저장
        )



        # 클래스 불균형을 반영하기 위한 변수
        y_train_binary = (y_train > 0.5).astype(int)
        positive_count = y_train_binary.sum()
        negative_count = len(y_train_binary) - positive_count
        
        pos_weight = torch.tensor(negative_count / positive_count)  # 기본
        
        # sqrt 적용:
        # pos_weight = torch.sqrt(torch.tensor(negative_count / positive_count))
        
        # # log 적용:
        # pos_weight = torch.log1p(torch.tensor(negative_count / positive_count))
        # ✅ custom loss: BCEWithLogitsLoss 
        custom_loss = nn.BCEWithLogitsLoss(pos_weight=pos_weight) # 클래스 비율을 반영

        # Trainer 생성: objective "binary"로 설정, 평가 지표로 Accuracy, AUROC, F1Score 사용
        trainer = Trainer(
            model=model, 
            objective="binary", ### 이거를 regression으로 바꾸는 게 나은 것인지를 모르겠음
            custom_loss_function=custom_loss,  # 기본 손실 함수 사용 (필요 시 사용자 정의 함수 가능)
            optimizers=optimizer,       # 옵티마이저 (기본값: Adam)
            initializers=None,          # 가중치 초기화 없음 (기본값)
            callbacks=[early_stopping, model_checkpoint],  # 조기 종료 (3 에포크 동안 개선 없으면 종료)
            metrics=[Accuracy()],
            verbose=1,                  # 학습 로그 출력 (기본값: 1)
            seed=seed                     # 랜덤 시드 설정 (기본값: 1)
        )
        
        # 학습: validation 데이터는 별도의 인자로 전달
        trainer.fit(
            X_train=X_train,  # ✅ Training 데이터를 딕셔너리로 
            n_epochs=100,  ####
            batch_size=1024, 
            X_val=X_val  # ✅ Validation 데이터를 딕셔너리로 전달
        )
        
        
        
        # 모델 학습 후 Validation 예측 코드:
        # 이미 확률이 계산되어 있는 컬럼을 사용합니다.
        valid_probs = trainer.predict_proba(X_wide=X_wide_valid, X_tab=X_tab_valid)[:,1] ####
        # valid_probs = trainer.predict(X_wide=X_wide_valid, X_tab=X_tab_valid)
        # valid_probs = np.clip(valid_probs, 0, 1) ##### regression 일때
        valid_pred = (valid_probs > 0.5).astype(int)
        
        

        # 실제 정답 
        y_valid = fold_valid['임신 성공 확률'].values.astype(int)
        

        
        # 대회 평가 지표
        fold_score = competition_metric(y_valid, valid_probs)
        fold_wbs = weighted_brier_score(y_valid, valid_probs)
        fold_f1 = f1_score(y_valid, valid_probs)

        
        print(f"Seed[{seed:<3}] Fold {fold + 1} | SCORE: {fold_score:.7f} | WBS: {fold_wbs:.7f} | F1: {fold_f1:.7f}")

        
        scores.append(fold_score)
        wbs_scores.append(fold_wbs)
        f1_scores.append(fold_f1)
        
        total_score.append(fold_score)
        total_wbs_score.append(fold_wbs)
        total_f1_score.append(f1_scores)
        
        # Test 데이터 예측 (각 fold의 모델로 예측한 결과 저장)
        test_pred = trainer.predict_proba(X_wide=X_wide_test, X_tab=X_tab_test)[:,1]
        # test_pred = trainer.predict(X_wide=X_wide_test, X_tab=X_tab_test)
        # test_pred = np.clip(test_pred, 0, 1) ##### regression 일때
        test_preds.append(test_pred)
    
    # Fold 별 평균 성능 출력
    avg_score = np.mean(scores)
    avg_wbs = np.mean(wbs_scores)
    avg_f1 = np.mean(f1_scores)
    
    print("-" * 80)
    print(f"Seed[{seed:<3}] Average Metrics | SCORE: {avg_score:.7f} | WBS: {avg_wbs:.7f} | F1: {avg_f1:.7f}")
    print("-" * 80)

# 전체 Validation 평균 성능 출력
val_score = np.mean(total_score)
val_wbs = np.mean(total_wbs_score)
val_f1 = np.mean(total_f1_score)


print("-" * 80)
print(f"Validation Average Metrics | SCORE: {val_score:.7f} | WBS: {val_wbs:.7f} | F1: {val_f1:.7f}")
print("-" * 80)

finish_time = time.time()
total_time = finish_time - start_time 

print(total_time)

In [ ]:
tmp_predictions['FastTabTransformer'] = np.mean(test_preds, axis=0)

## SoftVoting

In [ ]:
tmp_prediction

In [ ]:
final_prediction = (np.array(tmp_prediction['FTT']) + np.array(tmp_prediction['FastTabTransformer']))/2

## Submission

In [ ]:
sample_submission = pd.read_csv(sample_path)

sample_submission['임신 성공 확률'] = final_prediction

sample_submission



,ID,임신 성공 확률
0,TEST_00000,0.098759
1,TEST_00001,0.624725
2,TEST_00002,0.427147
3,TEST_00003,0.569354
4,TEST_00004,0.314667
...,...,...
54407,TEST_54407,0.710529
54408,TEST_54408,0.000194
54409,TEST_54409,0.632851
54410,TEST_54410,0.890026


In [ ]:
sample_submission.to_csv('./Submission/FTT_FastTabTrans_n_splits=5.csv', index=False)